<a href="https://colab.research.google.com/github/EA-b13/LULC_L3_Analysis/blob/main/Generate_NonPadded_LSMC_L9_TS_TrainingData%20(L7%2C8%2C9%2C%20S2%20clustering%20(C2)%2C%202018-19).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Coding Environment

## Library installations

In [ ]:
!pip install geemap geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.1 MB/s eta 0:00:00


## Module Imports

In [ ]:
import geemap
import geopandas as gpd
from pprint import pprint
import ee
from datetime import datetime, timedelta
import pandas as pd

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh

/content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh


## Authenticate to Google Earth Engine

In [ ]:
ee.Authenticate(force=True) #Uncomment this whenever needed, once done usually not needed for 1-2 days

In [ ]:
ee.Initialize(project = 'ee-indiasat')

# Function Definitions

## Defining constants

In [ ]:
chastainBandNames = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']

# Regression model parameters from Table-4. MSI TOA reflectance as a function of OLI TOA reflectance.
msiOLISlopes = [1.0946,1.0043,1.0524,0.8954,1.0049,1.0002]
msiOLIIntercepts = [-0.0107,0.0026,-0.0015,0.0033,0.0065,0.0046]

# Regression model parameters from Table-5. MSI TOA reflectance as a function of ETM+ TOA reflectance.
msiETMSlopes = [1.10601,0.99091,1.05681,1.0045,1.03611,1.04011]
msiETMIntercepts = [-0.0139,0.00411,-0.0024,-0.0076,0.00411,0.00861]

# Regression model parameters from Table-6. OLI TOA reflectance as a function of ETM+ TOA reflectance.
oliETMSlopes =[1.03501,1.00921,1.01991,1.14061,1.04351,1.05271];
oliETMIntercepts = [-0.0055,-0.0008,-0.0021,-0.0163,-0.0045,0.00261]

# Construct dictionary to handle all pairwise combos
chastainCoeffDict = { 'MSI_OLI':[msiOLISlopes,msiOLIIntercepts,1], # check what the third item corresponds to
                      'MSI_ETM':[msiETMSlopes,msiETMIntercepts,1],
                      'OLI_ETM':[oliETMSlopes,oliETMIntercepts,1],
                      'OLI_MSI':[msiOLISlopes,msiOLIIntercepts,0],
                      'ETM_MSI':[msiETMSlopes,msiETMIntercepts,0],
                      'ETM_OLI':[oliETMSlopes,oliETMIntercepts,0]
                    }

## Function defnitions

In [ ]:
'''
Function to mask cloudy pixels in Landsat-7
'''
# def maskL7cloud(image):
#   qa = image.select('BQA')
#   mask = qa.bitwiseAnd(1 << 4).eq(0)
#   return image.updateMask(mask).select(['B1', 'B2', 'B3' , 'B4' , 'B5' , 'B7']).rename('BLUE', 'GREEN', 'RED' , 'NIR' , 'SWIR1' , 'SWIR2')

'''
Function to mask cloudy pixels in Landsat-7 (C2)
'''
def maskL7cloud(image):
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(1 << 3).eq(0)
  return image.updateMask(mask).select(['B1', 'B2', 'B3' , 'B4' , 'B5' , 'B7']).rename('BLUE', 'GREEN', 'RED' , 'NIR' , 'SWIR1' , 'SWIR2')

'''
Function to mask cloudy pixels in Landsat-8
'''
# def maskL8cloud(image):
#   qa = image.select('BQA')
#   mask = qa.bitwiseAnd(1 << 4).eq(0)
#   return image.updateMask(mask).select(['B2', 'B3', 'B4' , 'B5' , 'B6' , 'B7']).rename('BLUE', 'GREEN', 'RED' , 'NIR' , 'SWIR1' , 'SWIR2')

'''
Function to mask cloudy pixels in Landsat-8 (C2)
'''
def maskL8cloud(image):
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(1 << 3).eq(0)
  return image.updateMask(mask).select(['B2', 'B3', 'B4' , 'B5' , 'B6' , 'B7']).rename('BLUE', 'GREEN', 'RED' , 'NIR' , 'SWIR1' , 'SWIR2')

'''
Function to mask cloudy pixels in Landsat-9 (C2)
'''
def maskL9cloud(image):
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(1 << 3).eq(0)
  return image.updateMask(mask).select(['B2', 'B3', 'B4' , 'B5' , 'B6' , 'B7']).rename('BLUE', 'GREEN', 'RED' , 'NIR' , 'SWIR1' , 'SWIR2')

'''
Function to mask clouds using the quality band of Semtinel-2 TOA
'''
def maskS2cloud(image):
  qa = image.select('QA60')
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
  return image.updateMask(mask).select(['B2', 'B3', 'B4', 'B8',  'B11', 'B12']).rename(['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])

'''
Function to mask clouds using the quality band of NASA-HLSL30 TOA
'''
def maskHLSL30cloud(image):
    # Select the Fmask band
    fmask_band = image.select(['Fmask'])
    # Create a mask for clear pixels (Fmask value of 0)
    clear_mask = fmask_band.bitwiseAnd(1 << 1).eq(0)  # Bit 1: Cloud
    # Update the image mask
    return image.updateMask(clear_mask).select(['B2', 'B3', 'B4', 'B5',  'B6', 'B7']).rename(['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])


'''
Get Landsat and Sentinel image collections
'''
def Get_L7_L8_L9_S2_ImageCollections(inputStartDate, inputEndDate, roi_boundary):
  # ------ Landsat 7 TOA
  L7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_TOA') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary) \
          .map(maskL7cloud)
  # print('\n Original Landsat 7 TOA dataset: \n',L7.limit(1).getInfo())
  # print('Number of images in Landsat 7 TOA dataset: \t',L7.size().getInfo())

  # ------ Landsat 8 TOA
  L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary) \
          .map(maskL8cloud)
  # print('\n Original Landsat 8 TOA dataset: \n', L8.limit(1).getInfo())
  # print('Number of images in Landsat 8 TOA dataset: \t',L8.size().getInfo())

  # ------ Sentinel-2 TOA
  S2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary)  \
          .map(maskS2cloud)
  # print('\n Original Sentinel-2 TOA dataset: \n',S2.limit(1).getInfo())
  # print('Number of images in Sentinel 2 TOA dataset: \t',S2.size().getInfo())
  # ------ Landsat 9 TOA
  L9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA').filterDate(inputStartDate, inputEndDate).filterBounds(roi_boundary).map(maskL9cloud);
  return L7, L8, L9, S2

'''
Get NASA-HLSL30 image collections
'''
def Get_HLSL30_ImageCollections(inputStartDate, inputEndDate, roi_boundary):
  HLSL30 = ee.ImageCollection('NASA/HLS/HLSL30/v002') \
          .filterDate(inputStartDate, inputEndDate) \
          .filterBounds(roi_boundary) \
          .map(maskHLSL30cloud)
  # print('\n Original Landsat 7 TOA dataset: \n',L7.limit(1).getInfo())
  # print('Number of images in Landsat 7 TOA dataset: \t',L7.size().getInfo())

  return HLSL30

'''
Function to apply model in one direction
'''
def dir0Regression(img,slopes,intercepts):
  return img.select(chastainBandNames).multiply(slopes).add(intercepts)


'''
Applying the model in the opposite direction
'''
def dir1Regression(img,slopes,intercepts):
  return img.select(chastainBandNames).subtract(intercepts).divide(slopes)


'''
Function to correct one sensor to another
'''
def harmonizationChastain(img, fromSensor,toSensor):
  # Get the model for the given from and to sensor
  comboKey = fromSensor.upper() + '_' + toSensor.upper()
  coeffList = chastainCoeffDict[comboKey]
  slopes = coeffList[0]
  intercepts = coeffList[1]
  direction = ee.Number(coeffList[2])

  # Apply the model in the respective direction
  out = ee.Algorithms.If(direction.eq(0),dir0Regression(img,slopes,intercepts),dir1Regression(img,slopes,intercepts))
  return ee.Image(out).copyProperties(img).copyProperties(img,['system:time_start'])


'''
Calibrate Landsat-8 (OLI) and Sentinel-2 (MSI) to Landsat-7 (ETM+)
'''
# def Harmonize_L7_L8_S2(L7, L8 = None, S2 = None):
#   # harmonization
#   harmonized_Landsat_ic = L7
#   if(L8 is not None):
#     harmonized_L8 = L8.map( lambda img: harmonizationChastain(img, 'OLI','ETM') )

#     # Merge harmonized landsat-8 and sentinel-2 to landsat-7 image collection
#     harmonized_Landsat_ic = ee.ImageCollection(L7.merge(harmonized_L8))
#   if(S2 is not None):
#     harmonized_S2 = S2.map( lambda img: harmonizationChastain(img, 'MSI','ETM') )
#     harmonized_LandsatSentinel_ic = ee.ImageCollection(harmonized_Landsat_ic.merge(harmonized_S2))
#     return harmonized_LandsatSentinel_ic
#   # print(harmonized_LandsatSentinel_ic.size().getInfo())
#   return harmonized_Landsat_ic

'''
Calibrate Landsat-8 (OLI), Landsat-9 (OLI-2) and Sentinel-2 (MSI) to Landsat-7 (ETM+)
'''

def Harmonize_L7_L8_L9_S2(L7, L8, S2 = None, L9 = None):
  # harmonization
  harmonized_L8 = L8.map( lambda img: harmonizationChastain(img, 'OLI','ETM') )

  # Merge harmonized landsat-8 and sentinel-2 to landsat-7 image collection
  harmonized_Landsat_ic = ee.ImageCollection(L7.merge(harmonized_L8))
  if(L9 is not None):
    harmonized_L9 = L9.map( lambda img: harmonizationChastain(img, 'OLI','ETM') )
    harmonized_Landsat_ic = ee.ImageCollection(harmonized_Landsat_ic.merge(harmonized_L9))
  if(S2 is not None):
    harmonized_S2 = S2.map( lambda img: harmonizationChastain(img, 'MSI','ETM') )
    harmonized_LandsatSentinel_ic = ee.ImageCollection(harmonized_Landsat_ic.merge(harmonized_S2))
    return harmonized_LandsatSentinel_ic
  # print(harmonized_LandsatSentinel_ic.size().getInfo())
  return harmonized_Landsat_ic


'''
Add NDVI band to harmonized image collection
'''
def addNDVI(image):
  return image.addBands(image.normalizedDifference(['NIR', 'RED']).rename('NDVI')).float()

'''
Add GCI band to harmonized image collection
'''
def addGCI(image):
  return image.addBands(image.normalizedDifference(['GREEN', 'RED']).rename('GCI')).float()

'''
Add NDVI band to harmonized image collection
'''
def addNDVIHLSL30(image):
    # Load bands with appropriate scale factors
    red = image.select('RED').multiply(0.0001)
    nir = image.select('NIR').multiply(0.0001)
    return image.addBands(image.normalizedDifference([nir, red]).rename('NDVI')).float()

'''
Function definitions to get NDVI values at each 16-day composites
'''
def Get_NDVI_image_datewise(harmonized_LS_ic):
  def get_NDVI_datewise(date):
    return harmonized_LS_ic.select(['NDVI']) \
                            .filterDate(ee.Date(date), ee.Date(date).advance(16, 'day')) \
                            .median() \
                            .set('system:time_start',ee.Date(date).millis())
  return get_NDVI_datewise

def Get_LS_16Day_NDVI_TimeSeries(inputStartDate, inputEndDate, harmonized_LS_ic):
  startDate = datetime.strptime(inputStartDate,"%Y-%m-%d")
  endDate = datetime.strptime(inputEndDate,"%Y-%m-%d")

  date_list = pd.date_range(start=startDate, end=endDate, freq='16D').tolist()
  date_list = ee.List( [datetime.strftime(curr_date,"%Y-%m-%d") for curr_date in date_list] )

  LSC =  ee.ImageCollection.fromImages(date_list.map(Get_NDVI_image_datewise(harmonized_LS_ic)))

  return LSC

'''
Function definitions to get GCI values at each 16-day composites
'''
def Get_GCI_image_datewise(harmonized_LS_ic):
  def get_GCI_datewise(date):
    return harmonized_LS_ic.select(['GCI']) \
                            .filterDate(ee.Date(date), ee.Date(date).advance(16, 'day')) \
                            .median() \
                            .set('system:time_start',ee.Date(date).millis())
  return get_GCI_datewise

def Get_LS_16Day_GCI_TimeSeries(inputStartDate, inputEndDate, harmonized_LS_ic):
  startDate = datetime.strptime(inputStartDate,"%Y-%m-%d")
  endDate = datetime.strptime(inputEndDate,"%Y-%m-%d")

  date_list = pd.date_range(start=startDate, end=endDate, freq='16D').tolist()
  date_list = ee.List( [datetime.strftime(curr_date,"%Y-%m-%d") for curr_date in date_list] )

  LSC =  ee.ImageCollection.fromImages(date_list.map(Get_NDVI_image_datewise(harmonized_LS_ic)))

  return LSC


'''
Pair available LSC and modis values for each time stamp.
'''
def pairLSModis(lsRenameBands):
  def pair(feature):
    date = ee.Date( feature.get('system:time_start') )
    startDateT = date.advance(-8,'day')
    endDateT = date.advance(8,'day')

    # ------ MODIS VI ( We can add EVI to the band list later )
    modis = ee.ImageCollection('MODIS/061/MOD13Q1') \
              .filterDate(startDateT, endDateT) \
              .select(['NDVI','SummaryQA']) \
              .filterBounds(roi_boundary) \
              .median() \
              .rename(['NDVI_modis', 'SummaryQA_modis'])

    return feature.rename(lsRenameBands).addBands(modis)
  return pair


'''
Function to get Pearson Correlation Coffecient to perform GapFilling
'''
def get_Pearson_Correlation_Coefficients(LSC_modis_paired_ic, roi_boundary, bandList):
  corr = LSC_modis_paired_ic.filterBounds(roi_boundary) \
                            .select(bandList).toArray() \
                            .arrayReduce( reducer = ee.Reducer.pearsonsCorrelation(), axes=[0], fieldAxis=1 ) \
                            .arrayProject([1]).arrayFlatten([['c', 'p']])
  return corr


'''
Fill gaps in LSC timeseries using modis data
'''
def gapfillLSM(LSC_modis_regression_model, LSC_bandName, modis_bandName):
  def peformGapfilling(image):
    offset = LSC_modis_regression_model.select('offset')
    scale = LSC_modis_regression_model.select('scale')
    nodata = -1

    lsc_image = image.select(LSC_bandName)
    modisfit = image.select(modis_bandName).multiply(scale).add(offset)

    mask = lsc_image.mask()#update mask needs an input (no default input from the API document)
    gapfill = lsc_image.unmask(nodata)
    gapfill = gapfill.where(mask.Not(), modisfit)

    '''
    in SummaryQA,
    0: Good data, use with confidence
    1: Marginal data, useful but look at detailed QA for more information
    2: Pixel covered with snow/ice
    3: Pixel is cloudy
    '''
    qc_m = image.select('SummaryQA_modis').unmask(3)  # missing value is grouped as cloud
    w_m  = modisfit.mask().rename('w_m').where(qc_m.eq(0), 0.8)  # default is 0.8
    w_m = w_m.where(qc_m.eq(1), 0.5)   # Marginal
    w_m = w_m.where(qc_m.gte(2), 0.2) # snow/ice or cloudy

    # make sure these modis values are read where there is missing data from LandSat, Sentinel
    w_l = gapfill.mask() # default is 1
    w_l = w_l.where(mask.Not(), w_m)

    return gapfill.addBands(w_l).rename(['gapfilled_'+LSC_bandName,'SummaryQA']) #have NDVI from modis and a summary of clarity for each

  return peformGapfilling


'''
Function to combine LSC with Modis data
'''
def Combine_LS_Modis(LSC):
  lsRenameBands = ee.Image(LSC.first()).bandNames().map( lambda band: ee.String(band).cat('_lsc') )
  LSC_modis_paired_ic = LSC.map( pairLSModis(lsRenameBands) )

  # Output contains scale, offset i.e. two bands
  LSC_modis_regression_model_NDVI = LSC_modis_paired_ic.select(['NDVI_modis', 'NDVI_lsc']) \
                                                        .reduce(ee.Reducer.linearFit())

  corr_NDVI = get_Pearson_Correlation_Coefficients(LSC_modis_paired_ic, roi_boundary, ['NDVI_modis', 'NDVI_lsc'])
  LSMC_NDVI = LSC_modis_paired_ic.map(gapfillLSM(LSC_modis_regression_model_NDVI, 'NDVI_lsc', 'NDVI_modis'))

  return LSMC_NDVI


'''
Mask out low quality pixels
'''
def mask_low_QA(lsmc_image):
  low_qa = lsmc_image.select('SummaryQA').neq(0.2)
  return lsmc_image.updateMask(low_qa).copyProperties(lsmc_image, ['system:time_start'])


'''
Add image timestamp to each image in time series
'''
def add_timestamp(image):
  timeImage = image.metadata('system:time_start').rename('timestamp')
  timeImageMasked = timeImage.updateMask(image.mask().select(0))
  return image.addBands(timeImageMasked)


'''
Perform linear interpolation on missing values
'''
def performInterpolation(image):
  image = ee.Image(image)
  beforeImages = ee.List(image.get('before'))
  beforeMosaic = ee.ImageCollection.fromImages(beforeImages).mosaic()
  afterImages = ee.List(image.get('after'))
  afterMosaic = ee.ImageCollection.fromImages(afterImages).mosaic()

  # Interpolation formula
  # y = y1 + (y2-y1)*((t – t1) / (t2 – t1))
  # y = interpolated image
  # y1 = before image
  # y2 = after image
  # t = interpolation timestamp
  # t1 = before image timestamp
  # t2 = after image timestamp

  t1 = beforeMosaic.select('timestamp').rename('t1')
  t2 = afterMosaic.select('timestamp').rename('t2')
  t = image.metadata('system:time_start').rename('t')
  timeImage = ee.Image.cat([t1, t2, t])
  timeRatio = timeImage.expression('(t - t1) / (t2 - t1)', {
                  't': timeImage.select('t'),
                  't1': timeImage.select('t1'),
                  't2': timeImage.select('t2'),
              })

  interpolated = beforeMosaic.add((afterMosaic.subtract(beforeMosaic).multiply(timeRatio)))
  result = image.unmask(interpolated)
  fill_value = ee.ImageCollection([beforeMosaic, afterMosaic]).mosaic()
  result = result.unmask(fill_value)

  return result.copyProperties(image, ['system:time_start'])


def interpolate_timeseries(S1_TS):
  lsmc_masked = S1_TS.map(mask_low_QA)
  filtered = lsmc_masked.map(add_timestamp)

  # Time window in which we are willing to look forward and backward for unmasked pixel in time series
  timeWindow = 120

  # Define a maxDifference filter to find all images within the specified days. Convert days to milliseconds.
  millis = ee.Number(timeWindow).multiply(1000*60*60*24)
  # Filter says that pick only those timestamps which lie between the 2 timestamps not more than millis difference apart
  maxDiffFilter = ee.Filter.maxDifference(
                              difference = millis,
                              leftField = 'system:time_start',
                              rightField = 'system:time_start',
                            )

  # Filter to find all images after a given image. Compare the image's timstamp against other images.
  # Images ahead of target image should have higher timestamp.
  lessEqFilter = ee.Filter.lessThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Similarly define this filter to find all images before a given image
  greaterEqFilter = ee.Filter.greaterThanOrEquals(
                            leftField = 'system:time_start',
                            rightField = 'system:time_start'
                          )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter1 = ee.Filter.And( maxDiffFilter, lessEqFilter )
  join1 = ee.Join.saveAll(
                  matchesKey = 'after',
                  ordering = 'system:time_start',
                  ascending = False
          )
  join1Result = join1.apply(
                  primary = filtered,
                  secondary = filtered,
                  condition = filter1
                )

  # Apply first join to find all images that are after the target image but within the timeWindow
  filter2 = ee.Filter.And( maxDiffFilter, greaterEqFilter )
  join2 = ee.Join.saveAll(
                  matchesKey = 'before',
                  ordering = 'system:time_start',
                  ascending = True
          )
  join2Result = join2.apply(
                  primary = join1Result,
                  secondary = join1Result,
                  condition = filter2
                )

  interpolated_S1_TS = ee.ImageCollection(join2Result.map(performInterpolation))

  return interpolated_S1_TS

# Original Pan-India Cropland Time Series

In [ ]:
# L7, L8 time series only
croplands = ee.FeatureCollection('projects/ee-indiasat/assets/Polygon_Groundtruth/Cropland_groundtruth')
roi_boundary = croplands.geometry()

startDate = '2018-07-01'
endDate = '2019-06-30'

L7, L8, S2 = Get_L7_L8_S2_ImageCollections(startDate, endDate, roi_boundary)

harmonized_L_ic = Harmonize_L7_L8_S2(L7, L8)
harmonized_L_ic = harmonized_L_ic.map(addNDVI)
LC = Get_LS_16Day_NDVI_TimeSeries(startDate, endDate, harmonized_L_ic)
LMC_NDVI = Combine_LS_Modis(LC)
Interpolated_LMC_NDVI = interpolate_timeseries(LMC_NDVI)
final_LMC_NDVI_TS = Interpolated_LMC_NDVI.select(['gapfilled_NDVI_lsc']).toBands()
final_LMC_NDVI_TS = final_LMC_NDVI_TS.clip(roi_boundary)

train_set = final_LMC_NDVI_TS.sampleRegions(
  collection = croplands,
  scale = 30,
  geometries = False
)

task = ee.batch.Export.table.toDrive(
  collection = train_set,
  folder = 'Non_Padded_Original_TS',
  description = 'Non_Padded_Original_NDVI_TS_L7_L8',
  fileNamePrefix = 'Non_Padded_Original_NDVI_TS_L7_L8',
  fileFormat = 'CSV',
  )

task.start()


In [ ]:
# HLSL30 time series
croplands = ee.FeatureCollection('projects/ee-indiasat/assets/Polygon_Groundtruth/Cropland_groundtruth')
roi_boundary = croplands.geometry()

startDate = '2018-07-01'
endDate = '2019-06-30'

HLSL30 = Get_HLSL30_ImageCollections(startDate, endDate, roi_boundary)

HLSL30 = HLSL30.map(addNDVI)
HLS = Get_LS_16Day_NDVI_TimeSeries(startDate, endDate, HLSL30)
HLSM_NDVI = Combine_LS_Modis(HLS)
Interpolated_HLSM_NDVI = interpolate_timeseries(HLSM_NDVI)
final_HLSM_NDVI_TS = Interpolated_HLSM_NDVI.select(['gapfilled_NDVI_lsc']).toBands()
final_HLSM_NDVI_TS = final_HLSM_NDVI_TS.clip(roi_boundary)

train_set = final_HLSM_NDVI_TS.sampleRegions(
  collection = croplands,
  scale = 30,
  geometries = False
)

task = ee.batch.Export.table.toDrive(
  collection = train_set,
  folder = 'Non_Padded_Original_TS',
  description = 'Non_Padded_Original_NDVI_HLSM',
  fileNamePrefix = 'Non_Padded_Original_NDVI_TS_HLSM',
  fileFormat = 'CSV',
  )

task.start()

In [ ]:
# L7, L8, L9, S2 time series (C2)
croplands = ee.FeatureCollection('projects/ee-indiasat/assets/Polygon_Groundtruth/Cropland_groundtruth')
roi_boundary = croplands.geometry()

startDate = '2018-07-01'
endDate = '2019-06-30'

L7, L8, L9, S2 = Get_L7_L8_L9_S2_ImageCollections(startDate, endDate, roi_boundary)

harmonized_LS_ic = Harmonize_L7_L8_L9_S2(L7, L8, S2, L9)
harmonized_LS_ic = harmonized_LS_ic.map(addNDVI)
LSC = Get_LS_16Day_NDVI_TimeSeries(startDate, endDate, harmonized_LS_ic)
LSMC_NDVI = Combine_LS_Modis(LSC)
Interpolated_LSMC_NDVI = interpolate_timeseries(LSMC_NDVI)
final_LSMC_NDVI_TS = Interpolated_LSMC_NDVI.select(['gapfilled_NDVI_lsc']).toBands()
final_LSMC_NDVI_TS = final_LSMC_NDVI_TS.clip(roi_boundary)

train_set = final_LSMC_NDVI_TS.sampleRegions(
  collection = croplands,
  scale = 30,
  geometries = False
)

task = ee.batch.Export.table.toDrive(
  collection = train_set,
  folder = 'Non_Padded_Original_TS',
  description = 'Non_Padded_Original_NDVI_LSC_L9_C2_TS',
  fileNamePrefix = 'Non_Padded_Original_NDVI_LSC_L9_C2_TS',
  fileFormat = 'CSV',
  )

task.start()
